In [1]:
import sys
sys.path.append(r'C:\Proyectos\Falcon\Scripts')
from utils import Utils

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from utils import Utils


from utils import send_data_multiple_ubidots
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
import math
import psycopg2
from sqlalchemy import create_engine

Realizamos consulta SQL a la BD del VProuter. Es necesario estar conectado a la VPN o estar dentro de la red

In [3]:
# Cargar variables de entorno
load_dotenv()

# Obtener el directorio del proyecto
ProDir = os.getcwd()

# Obtener la fecha inicial de consulta
# Abrir el archivo en modo de lectura para consultar la última fecha en que se cargaron datos a Ubidots, esta fecha se convierte en la nueva fecha inicial de consulta
with open(os.path.join(ProDir, 'UF.txt'), 'r') as archivo:
    # Leer el contenido del archivo en un string
    fecha_inicial = archivo.read()


# Obtener la fecha final de consulta de la hora actual del sistema 
fecha_actual = datetime.now()


# Formatear la fecha y hora del sistema según el formato requerido para hacer la consulta a el VP
formato_deseado = "%Y-%m-%d %H:%M:%S"
#fecha_final = '2023-11-03 00:00:00.000 -05:00' #fecha estatica para pruebas
fecha_final = fecha_actual.strftime(formato_deseado)  + '.000 -05:00'


try:
    # Establecer la conexión a la base de datos
    connection = psycopg2.connect(
        host = os.getenv("_host"),
        user = os.getenv("_user"),
        password = os.getenv("_password"),
        database = os.getenv("_database"),
        port = os.getenv("_port")
    )
    print('Conexión exitosa, la consulta se realizará entre las fechas ' + fecha_inicial + ' y ' + str(fecha_final))

    # Crear un cursor para la conexión
    cursor = connection.cursor()

    # Nueva consulta SQL usando las variables de fecha
    query = f"""
        SELECT
            data.connected_id,
            data.input_id,
            data.data,
            data.created
        FROM
            data.daq_data AS data
        WHERE
            data.created > '{fecha_inicial}' AND data.created < '{fecha_final}'
        ORDER BY
            data.created;

    """

    # Ejecutar la nueva consulta SQL
    cursor.execute(query)

    # Obtener todas las filas resultantes
    rows = cursor.fetchall()

    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]

    # Crear un DataFrame de pandas
    df = pd.DataFrame(rows, columns=column_names)

    # Imprimir el DataFrame o realizar otras operaciones con él
    print(f"DataFrame de los registros de data.daq_data entre '{fecha_inicial}' y '{fecha_final}':")
    print(df)

except Exception as ex:
    print(ex)
    print('No se pudo realizar la consulta')

finally:
    # Cerrar la conexión, independientemente de si se realizó la conexión o no
    if connection:
        connection.close()
        print('Conexión cerrada')


Conexión exitosa, la consulta se realizará entre las fechas 2025-01-12 00:00:00.000 -05:00 y 2025-03-13 13:59:48.000 -05:00
DataFrame de los registros de data.daq_data entre '2025-01-12 00:00:00.000 -05:00' y '2025-03-13 13:59:48.000 -05:00':
         connected_id  input_id          data                   created
0                  38         1  9.246570e+03 2025-01-12 06:01:00+01:00
1                  41         2  3.410616e+00 2025-01-12 06:01:00+01:00
2                  37         2  5.345307e+00 2025-01-12 06:01:00+01:00
3                  39         3  3.725190e+01 2025-01-12 06:01:00+01:00
4                  39         1  1.045032e+04 2025-01-12 06:01:00+01:00
...               ...       ...           ...                       ...
2734657             2         4  2.172610e+06 2025-03-13 19:59:00+01:00
2734658             2         3  2.031080e+01 2025-03-13 19:59:00+01:00
2734659            10        15  1.106219e+01 2025-03-13 19:59:00+01:00
2734660            10        14  1.10

Se contruye diccionario que relaciona la información de la consulta, con los nombres asignados a las variables

In [4]:
variables ={
    (40, 2): 'vpm-14-pressure-psi',
    (40, 4): 'vpm-14-totalizer-mscf',
    (40, 1): 'vpm-14-flow-scfm',
    (40, 3): 'vpm-14-temperature',
    (41, 2): 'vpm-12-pressure-psi',
    (41, 4): 'vpm-12-totalizer-mscf',
    (41, 1): 'vpm-12-flow-scfm',
    (41, 3): 'vpm-12-temperature'

}


In [5]:
# Función para mapear el diccionario
def map_name(row):
    key = (row['connected_id'], row['input_id'])
    return variables.get(key, None)

# Crear la nueva columna 'name' utilizando la función map
df['name'] = df.apply(map_name, axis=1)

# Reemplazar los valores de -70 que envían los dewpoint
#df['data'] = df['data'].replace(-70, np.nan)


#df.dropna(inplace=True)

# Hacer las conversiones de fecha y hora correspondientes para Ubi
# Convertir la columna "created" a DateTime con el formato deseado
df['created'] = pd.to_datetime(df['created'], utc=True)
df['Date'] = pd.to_datetime(df['created'], format="%Y-%m-%d %H:%M:%S%z")

#Falcon_data['Date'] = Falcon_data['Date'].Falcon_data.tz_convert('Etc/GMT+1')
df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert('Etc/GMT+1')

In [6]:
df_data = df[['Date', 'name', 'data']]

In [7]:
df_data

,Date,name,data
0,2025-01-12 04:01:00-01:00,None,9.246570e+03
1,2025-01-12 04:01:00-01:00,vpm-12-pressure-psi,3.410616e+00
2,2025-01-12 04:01:00-01:00,None,5.345307e+00
3,2025-01-12 04:01:00-01:00,None,3.725190e+01
4,2025-01-12 04:01:00-01:00,None,1.045032e+04
...,...,...,...
2734657,2025-03-13 17:59:00-01:00,None,2.172610e+06
2734658,2025-03-13 17:59:00-01:00,None,2.031080e+01
2734659,2025-03-13 17:59:00-01:00,None,1.106219e+01
2734660,2025-03-13 17:59:00-01:00,None,1.100589e+01
